# Data Loading

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import os
import seaborn as sns
import matplotlib.font_manager as font_manager


# Get the current working directory (which should be the "Code" folder)
current_dir = os.getcwd()
print(current_dir)  # This should print the path to your "Code" folder

# Load the CSV file
data_path = r"../Data/Supplementary_Figure_S2\Combined_Values_Switching_Light_Experiments.csv"
file_path_full = os.path.abspath(data_path)

df_switch = pd.read_csv(file_path_full)


df_switch['speed (um/s)']=df_switch['speed'].copy() * 4.8 * 16 # pizel size * acquistion binning factor * image analysis binning factor

df_N_switch=df_switch.copy()
df_N_switch=df_N_switch[df_N_switch['Condition']=='Nimodipine']

df_C_switch=df_switch.copy()
df_C_switch=df_C_switch[df_C_switch['Condition']=='Control']

# Assuming your dataframe is named 'df'
max_speed_X = df_C_switch.groupby(['trial', 'row', 'column'])['speed (um/s)'].max().reset_index()
max_speed_X['Media']='Control'

median_speed_X = df_C_switch.groupby(['trial', 'row', 'column'])['speed (um/s)'].median().reset_index()
median_speed_X['Media']='Control'

mean_speed_X = df_C_switch.groupby(['trial', 'row', 'column'])['speed (um/s)'].mean().reset_index()
mean_speed_X['Media']='Control'

# Assuming your dataframe is named 'df'
max_speed_X_N = df_N_switch.groupby(['trial', 'row', 'column'])['speed (um/s)'].max().reset_index()
max_speed_X_N['Media']='Nimodipine'

median_speed_X_N = df_N_switch.groupby(['trial', 'row', 'column'])['speed (um/s)'].median().reset_index()
median_speed_X_N['Media']='Nimodipine'

mean_speed_X_N = df_N_switch.groupby(['trial', 'row', 'column'])['speed (um/s)'].mean().reset_index()
mean_speed_X_N['Media']='Nimodipine'

max_speed_combined=pd.concat([max_speed_X,max_speed_X_N])
max_speed_combined['Speed measurement']='Maximum Speed'

median_speed_combined=pd.concat([median_speed_X,median_speed_X_N])
median_speed_combined['Speed measurement']='Median Speed'

mean_speed_combined=pd.concat([mean_speed_X,mean_speed_X_N])
mean_speed_combined['Speed measurement']='Mean Speed'


both_speeds_combined=pd.concat([max_speed_combined,median_speed_combined])

all_speeds_combined=pd.concat([max_speed_combined,mean_speed_combined,median_speed_combined])


# Plotting Max Speed

In [ ]:
# Define font styles
titlefont = {'fontname': 'Candara', 'size': 18}
figurefont = {'fontname': 'Candara', 'size': 16}
tickfont = {'fontname': 'Candara', 'size': 14}

font = font_manager.FontProperties(family='Candara', math_fontfamily='custom', size=16)

# Add the font file
font_path = r"../Code/candara-font-family/Candara.ttf"
font_path_full = os.path.abspath(font_path)
font_manager.fontManager.addfont(font_path_full)

plt.rcParams['mathtext.fontset'] = 'custom' # supported values are ['dejavusans', 'dejavuserif', 'cm', 'stix', 'stixsans', 'custom']
plt.rcParams['mathtext.it'] = 'Candara:italic'

fig = plt.figure(figsize=(8, 6))

color_dict = {"Control": "turquoise", "Nimodipine": "orangered"}

ax=sns.boxplot(x=max_speed_combined['Media'] , y=max_speed_combined['speed (um/s)'], palette=color_dict)

ax.set_ylabel('Maximum Swimming Speed ($\mathit{\mu}$m/s)', **figurefont)

ax.set_xlabel('', **figurefont)

ax.tick_params(axis='both', which='major', labelsize=tickfont['size'], labelfontfamily=tickfont['fontname'])


# Set bar positions
bar_width = 0.35
r1 = 0 - bar_width
r2 = 1 + bar_width

sample_size_control=len(max_speed_X)
sample_size_nimodipine=len(max_speed_X_N)

# # Add sample sizes
ax.text(r1, 10, f'n={sample_size_control}', ha='center', va='bottom', fontsize=figurefont['size'],fontfamily=tickfont['fontname'])
ax.text(r2, 10, f'n={sample_size_nimodipine}', ha='center', va='bottom', fontsize=figurefont['size'],fontfamily=tickfont['fontname'])

# Perform t-tests and add significance stars

from scipy.stats import ttest_ind

# Perform t-tests
test_results = {}

measurement='Maximum Speed'
sub_df = all_speeds_combined[all_speeds_combined['Speed measurement'] == measurement]
media1_speeds = sub_df[sub_df['Media'] == 'Control']['speed (um/s)']
media2_speeds = sub_df[sub_df['Media'] == 'Nimodipine']['speed (um/s)']

t_stat, p_value = ttest_ind(media1_speeds, media2_speeds, equal_var=False)
test_results[measurement] = p_value

# Function to plot with significance
def plot_with_significance(ax, p_values):
    measurement='Maximum Speed'
    
    p_value = p_values[measurement]
    if p_value < 0.001:
        significance = '***'
    elif p_value < 0.01:
        significance = '**'
    elif p_value < 0.05:
        significance = '*'
    else:
        significance = 'ns'

    # Calculate position to place text
    sub_df = all_speeds_combined[all_speeds_combined['Speed measurement'] == measurement]
    max_speed = sub_df['speed (um/s)'].max()

    ax.annotate(significance, xy=(0.5, 0.90), xytext=(0.5, 0.90), xycoords='axes fraction', 
                fontsize=figurefont['size'],fontfamily=tickfont['fontname'], ha='center', va='bottom',
                arrowprops=dict(arrowstyle='-[, widthB=7, lengthB=0.5', lw=1.2, color='k'))

plot_with_significance(ax, test_results)

ax.set_ylim(0,900)
# plt.legend(loc='center right',prop={'family': figurefont['fontname'], 'size': figurefont['size']})


save_path = r"../Code/Graphs"
save_dir = os.path.abspath(save_path)

if not os.path.exists(save_dir):
    # Create the directory if it does not exist
    os.makedirs(save_dir)

filename=r'Nimodipine Swimming Maximum Speeds.png'

fig.savefig(os.path.join(save_dir, filename), dpi=600, bbox_inches='tight')
